In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa
import datetime as dt
from functools import reduce

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/macropy/core/hquotes.py
QUANTAXIS>> Finding macros in 'macropy.core.hquotes'
QUANTAXIS>> Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/macropy/core/quotes.py
QUANTAXIS>> Finding macros in 'macropy.core.quotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/macropy/core/failure.py
QUANTAXIS>> Finding macros in 'macropy.core.failure'
QUANTAXIS>> Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/cffi/api.py
QUANTAXIS>> Fi

no display found. Using non-interactive Agg backend
if you use ssh, you can use ssh with -X parmas to avoid this issue


QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/scipy/__config__.py
QUANTAXIS>> Finding macros in 'scipy.__config__'
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.68
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##```

In [42]:
def go_up(data, N=5, P=3, M=0.1):
    ind = (data.close > data.close.shift(1)) * 1
    checked = check(ind, -N)
    lift = data.high.rolling(N).max().shift(-N-1) / data.open.shift(-1)
    flat = ((data.close == data.open) & (data.close == data.high)) * 1    
    return pd.DataFrame({'GOUP': (~(flat > 0) & ~(flat.shift(-1) > 0) & (checked >= P) & (lift > (1+M))) * 1})


def check(signal, days):
    return reduce(lambda x, y: x + y, [signal.shift(x).fillna(0)
                                       for x in (range(days) if days >= 0 else range(-1, days-1, -1))])


In [ ]:
today = dt.datetime.today()
# today = dt.datetime(2018, 7, 6)
today_str = today.strftime('%Y-%m-%d')

# start_date = today - dt.timedelta(days=365)

stocks = qa.QA_fetch_stock_list_adv()
stock_list = stocks.code.tolist()
for stock in stock_list[:10]:
    print('handling %s' % stock)
    try:
        candles = qa.QA_fetch_stock_day_adv(stock, start='2014-01-01', end=today_str).to_qfq()
    except:
        print('data error during {}'.format(stock))
        continue

In [4]:
candles = qa.QA_fetch_stock_day_adv('000001', start='2014-01-01', end='2018-08-18').to_qfq()

In [43]:
ca = candles.add_func(go_up)

In [44]:
dd = candles.data
dd['rh'] = candles.data.high.rolling(5).max().shift(-6)
dd['goup'] = ca
dd['next'] = candles.data.open.shift(-1)
dd['upup'] = dd['rh'] / dd['next'] > 1.1
dd

,,open,high,low,close,volume,amount,preclose,adj,rh,goup,next,upup
date,code,,,,,,,,,,,,
2014-01-02,000001,6.556834,6.654213,6.518965,6.616344,9.055756e+05,5.962237e+08,NaN,0.540993,6.491915,0,6.573064,False
2014-01-03,000001,6.573064,6.578474,6.372897,6.454046,1.018708e+06,6.566313e+08,6.616344,0.540993,6.486505,0,6.432406,False
2014-01-06,000001,6.432406,6.491915,6.221419,6.313388,1.076018e+06,6.792804e+08,6.454046,0.540993,6.486505,0,6.237649,False
2014-01-07,000001,6.237649,6.362077,6.226829,6.291748,6.255294e+05,3.939776e+08,6.313388,0.540993,6.486505,0,6.297158,False
2014-01-08,000001,6.297158,6.464866,6.237649,6.362077,8.461626e+05,5.384362e+08,6.291748,0.540993,6.464866,0,6.324207,False
2014-01-09,000001,6.324207,6.486505,6.302568,6.394537,8.974572e+05,5.768705e+08,6.362077,0.540993,6.443226,0,6.372897,False
2014-01-10,000001,6.372897,6.464866,6.307978,6.394537,7.046247e+05,4.504877e+08,6.394537,0.540993,6.383717,0,6.383717,False
2014-01-13,000001,6.383717,6.443226,6.216009,6.275518,8.851779e+05,5.593752e+08,6.394537,0.540993,6.383717,0,6.259288,False
2014-01-14,000001,6.259288,6.372897,6.194369,6.340437,6.699293e+05,4.207153e+08,6.275518,0.540993,6.416176,0,6.329617,False


In [45]:
dd.loc[dd.goup > 0, :].shape

(0, 12)

In [49]:
 
dd['skdj.k.9.3'], dd['skdj.d.9.3'] = candles.add_func(qa.QA_indicator_SKDJ, N=9, M=3).loc[:, ['SKDJ_K', 'SKDJ_D']]
dd

,,open,high,low,close,volume,amount,preclose,adj,rh,goup,next,upup,skdj.k.9.3,skdj.d.9.3
date,code,,,,,,,,,,,,,,
2014-01-02,000001,6.556834,6.654213,6.518965,6.616344,9.055756e+05,5.962237e+08,NaN,0.540993,6.491915,0,6.573064,False,SKDJ_K,SKDJ_D
2014-01-03,000001,6.573064,6.578474,6.372897,6.454046,1.018708e+06,6.566313e+08,6.616344,0.540993,6.486505,0,6.432406,False,SKDJ_K,SKDJ_D
2014-01-06,000001,6.432406,6.491915,6.221419,6.313388,1.076018e+06,6.792804e+08,6.454046,0.540993,6.486505,0,6.237649,False,SKDJ_K,SKDJ_D
2014-01-07,000001,6.237649,6.362077,6.226829,6.291748,6.255294e+05,3.939776e+08,6.313388,0.540993,6.486505,0,6.297158,False,SKDJ_K,SKDJ_D
2014-01-08,000001,6.297158,6.464866,6.237649,6.362077,8.461626e+05,5.384362e+08,6.291748,0.540993,6.464866,0,6.324207,False,SKDJ_K,SKDJ_D
2014-01-09,000001,6.324207,6.486505,6.302568,6.394537,8.974572e+05,5.768705e+08,6.362077,0.540993,6.443226,0,6.372897,False,SKDJ_K,SKDJ_D
2014-01-10,000001,6.372897,6.464866,6.307978,6.394537,7.046247e+05,4.504877e+08,6.394537,0.540993,6.383717,0,6.383717,False,SKDJ_K,SKDJ_D
2014-01-13,000001,6.383717,6.443226,6.216009,6.275518,8.851779e+05,5.593752e+08,6.394537,0.540993,6.383717,0,6.259288,False,SKDJ_K,SKDJ_D
2014-01-14,000001,6.259288,6.372897,6.194369,6.340437,6.699293e+05,4.207153e+08,6.275518,0.540993,6.416176,0,6.329617,False,SKDJ_K,SKDJ_D


In [50]:
dd.amount / dd.volume

date        code  
2014-01-02  000001    658.392022
2014-01-03  000001    644.572438
2014-01-06  000001    631.291063
2014-01-07  000001    629.830595
2014-01-08  000001    636.327049
2014-01-09  000001    642.783341
2014-01-10  000001    639.330027
2014-01-13  000001    631.935298
2014-01-14  000001    627.999534
2014-01-15  000001    629.488830
2014-01-16  000001    632.065962
2014-01-17  000001    622.631441
2014-01-20  000001    614.837979
2014-01-21  000001    617.899170
2014-01-22  000001    632.258211
2014-01-23  000001    633.136156
2014-01-24  000001    632.666301
2014-01-27  000001    620.794221
2014-01-28  000001    619.473173
2014-01-29  000001    626.745249
2014-01-30  000001    617.488244
2014-02-07  000001    607.372581
2014-02-10  000001    623.630012
2014-02-11  000001    647.951049
2014-02-12  000001    647.779388
2014-02-13  000001    657.516751
2014-02-14  000001    652.070430
2014-02-17  000001    646.433807
2014-02-18  000001    635.552653
2014-02-19  000001    64

In [51]:
candles.data

,,open,high,low,close,volume,amount,preclose,adj,rh,goup,next,upup,skdj.k.9.3,skdj.d.9.3
date,code,,,,,,,,,,,,,,
2014-01-02,000001,6.556834,6.654213,6.518965,6.616344,9.055756e+05,5.962237e+08,NaN,0.540993,6.491915,0,6.573064,False,SKDJ_K,SKDJ_D
2014-01-03,000001,6.573064,6.578474,6.372897,6.454046,1.018708e+06,6.566313e+08,6.616344,0.540993,6.486505,0,6.432406,False,SKDJ_K,SKDJ_D
2014-01-06,000001,6.432406,6.491915,6.221419,6.313388,1.076018e+06,6.792804e+08,6.454046,0.540993,6.486505,0,6.237649,False,SKDJ_K,SKDJ_D
2014-01-07,000001,6.237649,6.362077,6.226829,6.291748,6.255294e+05,3.939776e+08,6.313388,0.540993,6.486505,0,6.297158,False,SKDJ_K,SKDJ_D
2014-01-08,000001,6.297158,6.464866,6.237649,6.362077,8.461626e+05,5.384362e+08,6.291748,0.540993,6.464866,0,6.324207,False,SKDJ_K,SKDJ_D
2014-01-09,000001,6.324207,6.486505,6.302568,6.394537,8.974572e+05,5.768705e+08,6.362077,0.540993,6.443226,0,6.372897,False,SKDJ_K,SKDJ_D
2014-01-10,000001,6.372897,6.464866,6.307978,6.394537,7.046247e+05,4.504877e+08,6.394537,0.540993,6.383717,0,6.383717,False,SKDJ_K,SKDJ_D
2014-01-13,000001,6.383717,6.443226,6.216009,6.275518,8.851779e+05,5.593752e+08,6.394537,0.540993,6.383717,0,6.259288,False,SKDJ_K,SKDJ_D
2014-01-14,000001,6.259288,6.372897,6.194369,6.340437,6.699293e+05,4.207153e+08,6.275518,0.540993,6.416176,0,6.329617,False,SKDJ_K,SKDJ_D


In [53]:
pd.concat([dd.loc[:, ['open', 'high']], dd.loc[:, ['low', 'volume']]], axis=1)

,,open,high,low,volume
date,code,,,,
2014-01-02,000001,6.556834,6.654213,6.518965,9.055756e+05
2014-01-03,000001,6.573064,6.578474,6.372897,1.018708e+06
2014-01-06,000001,6.432406,6.491915,6.221419,1.076018e+06
2014-01-07,000001,6.237649,6.362077,6.226829,6.255294e+05
2014-01-08,000001,6.297158,6.464866,6.237649,8.461626e+05
2014-01-09,000001,6.324207,6.486505,6.302568,8.974572e+05
2014-01-10,000001,6.372897,6.464866,6.307978,7.046247e+05
2014-01-13,000001,6.383717,6.443226,6.216009,8.851779e+05
2014-01-14,000001,6.259288,6.372897,6.194369,6.699293e+05


In [54]:
A = dd.close * 2
B = dd.close - dd.open

In [56]:
qa.IF((dd.close > dd.open) & (dd.close> 0) , A, B)

date        code  
2014-01-02  000001    13.232687
2014-01-03  000001    -0.119018
2014-01-06  000001    -0.119018
2014-01-07  000001    12.583496
2014-01-08  000001    12.724154
2014-01-09  000001    12.789073
2014-01-10  000001    12.789073
2014-01-13  000001    -0.108199
2014-01-14  000001    12.680874
2014-01-15  000001    -0.016230
2014-01-16  000001    12.648415
2014-01-17  000001    -0.075739
2014-01-20  000001    -0.097379
2014-01-21  000001    12.291360
2014-01-22  000001    12.756613
2014-01-23  000001    -0.037870
2014-01-24  000001    12.615955
2014-01-27  000001    -0.091969
2014-01-28  000001    12.442838
2014-01-29  000001    12.507757
2014-01-30  000001    -0.081149
2014-02-07  000001    12.312999
2014-02-10  000001    12.518577
2014-02-11  000001    12.983831
2014-02-12  000001    12.918911
2014-02-13  000001    13.221867
2014-02-14  000001    -0.016230
2014-02-17  000001    -0.070329
2014-02-18  000001    -0.135248
2014-02-19  000001    12.983831
                     

In [57]:
dd.rename?

In [59]:
ds = dd.loc[:, ['low', 'high']].reset_index()
ds.loc[ds.date > '2018-09-01', :].to_hdf('../data/test.hdf', key='test')

In [61]:
ds.pop?

In [64]:
dm = ds.max()
dm.loc[dm > 10]

TypeError: Cannot compare type 'Timestamp' with type 'int'

In [86]:
train = pd.read_hdf('../data/inds/000001.hdf', key='train')

In [87]:
tm = train.drop(['code', 'date'], axis=1).max()

In [89]:
tm.loc[(tm > 10000) & (tm > 0)]

vr_3              inf
vr_6     2.481392e+17
vr_9     8.947633e+16
vr_12    1.947017e+17
br_3              inf
cr_3              inf
dtype: float64

In [74]:
test.loc[test.date > '2018-03-10', ['date', 'boll_lb_26']]

,date,boll_lb_26
1104,2018-03-12,-14.674913
1105,2018-03-13,-15.491138
1106,2018-03-14,-16.047363
1107,2018-03-15,-17.653271
1108,2018-03-16,-21.033246
1109,2018-03-19,-31.738922
1110,2018-03-20,674852.149021
1111,2018-03-21,33.254144
1112,2018-03-22,27.623634
1113,2018-03-23,29.998400


In [72]:
dd.reset_index().loc[dd.reset_index().date > '2018-03-10', :].head(50)

,date,code,open,high,low,close,volume,amount,preclose,adj,rh,goup,next,upup,skdj.k.9.3,skdj.d.9.3
1020,2018-03-12,000001,11.961800,11.981490,11.764897,11.843658,1.288662e+06,1.526643e+09,11.902729,0.98451,11.814123,0,11.853503,False,SKDJ_K,SKDJ_D
1021,2018-03-13,000001,11.853503,12.030715,11.814123,11.833813,1.099295e+06,1.309543e+09,11.843658,0.98451,11.932264,0,11.794433,False,SKDJ_K,SKDJ_D
1022,2018-03-14,000001,11.794433,11.814123,11.646756,11.735362,6.455941e+05,7.570908e+08,11.833813,0.98451,11.932264,0,11.607376,False,SKDJ_K,SKDJ_D
1023,2018-03-15,000001,11.607376,11.666446,11.479390,11.528615,1.173877e+06,1.356879e+09,11.735362,0.98451,11.932264,0,11.538460,False,SKDJ_K,SKDJ_D
1024,2018-03-16,000001,11.538460,11.666446,11.459699,11.459699,9.781341e+05,1.130784e+09,11.528615,0.98451,11.932264,0,11.479390,False,SKDJ_K,SKDJ_D
1025,2018-03-19,000001,11.479390,11.656601,11.430164,11.646756,8.212591e+05,9.476648e+08,11.459699,0.98451,11.932264,0,11.558150,False,SKDJ_K,SKDJ_D
1026,2018-03-20,000001,11.558150,11.695982,11.538460,11.636911,7.883615e+05,9.168585e+08,11.646756,0.98451,11.784588,0,11.764897,False,SKDJ_K,SKDJ_D
1027,2018-03-21,000001,11.764897,11.932264,11.666446,11.715672,1.467846e+06,1.736437e+09,11.636911,0.98451,11.174191,0,11.715672,False,SKDJ_K,SKDJ_D
1028,2018-03-22,000001,11.715672,11.784588,11.440009,11.479390,9.997641e+05,1.155741e+09,11.715672,0.98451,11.026515,0,11.075740,False,SKDJ_K,SKDJ_D
1029,2018-03-23,000001,11.075740,11.174191,10.750852,11.164346,1.854414e+06,2.037734e+09,11.479390,0.98451,10.996979,0,10.977289,False,SKDJ_K,SKDJ_D
